In [109]:
import os
os.startfile('take.exe')

In [105]:
import numpy as np

def gauss_jordan(matrix): #calcula a inversa
    rows, cols = matrix.shape
    identity = np.identity(rows)
    augmented_matrix = np.concatenate((matrix, identity), axis=1)

    for i in range(rows):
        pivot_row = i
        while pivot_row < rows and augmented_matrix[pivot_row, i] == 0:
            pivot_row += 1
        if pivot_row == rows:
            raise ValueError("Nao tem inversa")
        augmented_matrix[[i, pivot_row]] = augmented_matrix[[pivot_row, i]]
        pivot = augmented_matrix[i, i]
        augmented_matrix[i] /= pivot
        for j in range(rows):
            if j != i:
                factor = augmented_matrix[j, i]
                augmented_matrix[j] -= factor * augmented_matrix[i]
    return augmented_matrix[:, cols:]

def matrix_multiply(A, B): #multiplicacao entre matriz
    rows_A, cols_A = A.shape
    rows_B, cols_B = B.shape

    if cols_A != rows_B:
        raise ValueError("Número de colunas da primeira matriz deve ser igual ao número de linhas da segunda matriz.")

    # Inicializar a matriz resultado com zeros
    result = np.zeros((rows_A, cols_B))

    # Realizar a multiplicação de matrizes
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i, j] += A[i, k] * B[k, j]

    return result

def simplex_method(c, A, b):

    basic_vars = list(range(num_vars, num_vars + num_constraints))
    non_basic_vars = list(range(num_vars))

    # Iterar até encontrar a solução ótima (Parte da Fase I e Fase II)
    while True:

        B = A[:, basic_vars]
        N = A[:, non_basic_vars]
        c_B = c[basic_vars]
        c_N = c[non_basic_vars]


        B_inv = gauss_jordan(B)
        x_B = matrix_multiply(B_inv, b.reshape(-1, 1)).flatten() #passo 1 calculo da solucao basica

        # Passo 2: calcular custos relativos (tanto fas e I e II)
        #2.1
        lambda_T = matrix_multiply(c_B.reshape(1, -1), B_inv).flatten()

        #2.2
        reduced_costs = c_N - matrix_multiply(lambda_T.reshape(1, -1), N).flatten()

        #3 otimalidade
        # Teste de otimalidade Fase I se cnk >=0 entao
        if all(rc >= 0 for rc in reduced_costs): #verifica se todos os custos sao positivos
            solution = np.zeros(len(c)) #cria um vetor do tamanho de c comm zeros 0
            for i, var in enumerate(basic_vars):
                solution[var] = x_B[i] #Preenche o vetor solution com os valores das variáveis básicas da solução atual (atualiza a solucao otima, com os novos valores das var basicas)
            optimal_value = np.dot(c, solution) #calulca na funcao obj: o valor ótimo multiplicando a função objetivo c pela solução .
            if maximize:
                optimal_value = -optimal_value #transforma pra max dnv
            return solution[:num_vars], optimal_value  #

        # Determinar a variável a entrar na base (Parte da Fase I e Fase II)
        entering_var = non_basic_vars[np.argmin(reduced_costs)]

        # Calcular direção simplex (Parte da Fase I e Fase II)
        y = matrix_multiply(B_inv, A[:, entering_var].reshape(-1, 1)).flatten()


 
        if all(y <= 0):
            raise Exception("Problema ilimitado")
        2.3
        # Determinar a variável a sair da base (fase 1 e 2)
        ratios = np.array([x_B[i] / y[i] if y[i] > 0 else np.inf for i in range(len(y))]) #lista de valores minimos para cada indice i onde o y é positivo
        leaving_var_index = np.argmin(ratios) #determina qual indice da variavel que deve sair da base (np.argmin retorna o menor valor)
        leaving_var = basic_vars[leaving_var_index] #retirando do vetor das var basicas o menor valor

        # att as variáveis básicas e não-básicas (fase 1 e 2)
        basic_vars[leaving_var_index] = entering_var #att a base que ta saindo pela que entrando (nova basee)
        non_basic_vars[non_basic_vars.index(entering_var)] = leaving_var #att as var nao basica



global num_constraints
global num_vars
global maximize
b = np.loadtxt('b.txt')
c = np.loadtxt('c.txt')
A = np.loadtxt('A.txt') 
arr = np.loadtxt('infos.txt')
maximize = bool(arr[0])
num_constraints = int(arr[1])
num_vars = int(arr[2])
solution, optimal_value = simplex_method(c, A, b)
print("Solução:", solution)
print("Valor ótimo:", optimal_value)


Solução: [6.4 5.2]
Valor ótimo: 11.6
